In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import casadi as ca
import BF_PCA
import time

import sys
sys.path.insert(0, '../')
from pyecca.lie import se3, so3, matrix_lie_group

from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib.text import Annotation


In [2]:
class Arrow3D(FancyArrowPatch):

    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs) 
    
def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)

setattr(Axes3D, 'arrow3D', _arrow3D)

class Annotation3D(Annotation):

    def __init__(self, text, xyz, *args, **kwargs):
        super().__init__(text, xy=(0, 0), *args, **kwargs)
        self._xyz = xyz

    def draw(self, renderer):
        x2, y2, z2 = proj_transform(*self._xyz, self.axes.M)
        self.xy = (x2, y2)
        super().draw(renderer)



def _annotate3D(ax, text, xyz, *args, **kwargs):
    '''Add anotation `text` to an `Axes3d` instance.'''

    annotation = Annotation3D(text, xyz, *args, **kwargs)
    ax.add_artist(annotation)

setattr(Axes3D, 'annotate3D', _annotate3D)


In [3]:
def Ad(T):
    C = T[:3,:3]
    r = T[:3,3]
    return ca.vertcat(ca.horzcat(C, SO3.wedge(r)@C), ca.horzcat(ca.SX.zeros(3,3),C))

def barfoot_solve(Top, p, y, assoc, weight):
    #the incorporated weights assume that every landmark is observed len(y) = len(w) = len(p)
    Tau = Ad(Top)
    Cop = Top[:3,:3]
    rop = (-Cop.T@Top[:3,3])
    
    w = np.sum(weight)
    P = ca.SX(np.average(p,axis=0, weights=weight))
    Y = ca.SX(np.average(y,axis=0, weights=weight))
    
    I = 0
    for j in range(len(p)):
        pint0=(p[j] - P)
        wj = weight[j]
        I += wj*SO3.wedge(pint0)@SO3.wedge(pint0)
    I=-I/w
    
    M1 = ca.vertcat(ca.horzcat(ca.SX.eye(3), ca.SX.zeros(3,3)), ca.horzcat(SO3.wedge(P),ca.SX.eye(3)))
    M2 = ca.vertcat(ca.horzcat(ca.SX.eye(3), ca.SX.zeros(3,3)), ca.horzcat(ca.SX.zeros(3,3),I))
    M3 = ca.vertcat(ca.horzcat(ca.SX.eye(3), -SO3.wedge(P)), ca.horzcat(ca.SX.zeros(3,3),ca.SX.eye(3)))
    M=M1@M2@M3
    
    W = 0
    for j in range(len(y)):
        li = assoc[j]
        pj = p[li]
        yj = y[j]
        wj = weight[li]
        
        W += wj*(yj-Y)@(pj-P).T  
    W = W/w
    
    b=ca.SX.zeros(1,3)
    b[0] = ca.trace(SO3.wedge([1,0,0])@Cop@W.T)
    b[1] = ca.trace(SO3.wedge([0,1,0])@Cop@W.T)
    b[2] = ca.trace(SO3.wedge([0,0,1])@Cop@W.T) 

    a=ca.vertcat(Y-Cop@(P-rop),b.T-SO3.wedge(Y)@Cop@(P-rop))
    
    #Optimizied pertubation point
    eopt=Tau@ca.inv(M)@Tau.T@a
    
    return eopt

In [20]:
t0 = time.time()

#------------------------
# Define constant

deg_to_rad = np.pi / 180

noise={
    'odom_std': 0.0,
    'x_std': 0.05,
    'y_std': 0.0,
    'z_std': 0.0,
}

#------------------------
# Initialize states

#input pair: [dist , yaw                ,  pitch              ,  roll]
u = np.array([[3   , 0                  , 0                   ,    0],
             [5    , 53 * deg_to_rad    , 0      ,              0],
             [1    , 20 * deg_to_rad    , 0                   ,    0],
             [2    , 17 * deg_to_rad    , 0                   ,    0],
             [4    , 20 * deg_to_rad    , 0                   ,    0],])
             # [5    , 0 * deg_to_rad    , 0                   ,    0],
             # [3    , 60 * deg_to_rad    , 0                   ,    0],
             # [2    , 20 * deg_to_rad    , 0                   ,    0],
             # [4    , 30 * deg_to_rad    , 0                   ,    0],
             # [3    , 30 * deg_to_rad    , 0                   ,    0],])

#Time array 
dt = 1
tf = len(u) * dt

#Define landmarks and coresponding weight
landmarks = np.array([[1,1,0],
                     [3,4,0],
                     [6,5,0],
                     [3,8,0],
                     [2,6,0]])
weight = np.array([1,
                   1,
                   1,
                   1,
                   1])

#Define true state variable x described in inertial frame
x0 = np.array([0.,0.,0.]);
x_prev = x0;
Rot0 = np.identity(3)

#Define estimated state wrt inetial frame
xh = np.array([x0])
Roth = Rot0
Rotk = Rot0

#Initialize Lie Group
SE3 = se3._SE3()
SO3 = so3._Dcm()

J = ca.SX.zeros(1,1)

hist = {'x': [x0],
        'y': [],
        'R':  [Rot0],
        'T': [],
        'assoc':[],
       'Frame': ['inertial']};
Top_allbody = []
yk_allbody = []
assoc_allbody = []


#------------------------
#Begin Simulation

t_vect = np.arange(0,tf,dt)
for k, tk in enumerate(t_vect):
    
    # print("iteration",k)
    
    # !!! Move then measure !!!
    
    #Assign Rotation and translation Input
    # !!! rotate then translate !!!
    uk = u[k]
    
    yawk = uk[1]
    pitchk = uk[2]
    rollk = uk[3]
    
    Rot_prev2body   = BF_PCA.euler2rot(yawk, pitchk, rollk) #previous body to current body
    Rotk = Rotk@Rot_prev2body #inertial to current body
    
    uTrans_k = Rotk@np.array([uk[0], 0, 0]).T
    
    #Update State variables (x is in inertial frame) xk = r_vk_i (barfoot)
    xk = x_prev + uTrans_k; 
    
    #Measure landmarks
    yk = BF_PCA.measure_landmark(Rotk, xk, landmarks, noise=noise, range_max=1000)
    yk_allbody.append(yk)
    
    #Transformation Matrix (True)
    Tk = np.vstack([np.hstack([Rotk.T, np.matmul(-Rotk.T,xk).reshape(3,1)]), np.array([[0, 0, 0, 1]])])
    
    #Define symbolic perturbation of T
    trans_sym = ca.vertcat(ca.SX.sym('u_'+str(k)), ca.SX.sym('v_'+str(k)), ca.SX.sym('w_'+str(k)))
    angles_sym = ca.vertcat(ca.SX.sym('omega1_'+str(k)), ca.SX.sym('omega2_'+str(k)), ca.SX.sym('omega3_'+str(k)))
    epsilon = ca.vertcat(trans_sym, angles_sym)
    if k == 0:
        epsilon_all = epsilon
    else:
        epsilon_all = ca.vertcat(epsilon_all, epsilon)

    #Associate landmarks index to measurement observed using maximum likelihood
    assoc = [ BF_PCA.data_association(xk, yi, landmarks, Rotk) for yi in yk ]
    assoc_allbody.append(assoc)
    
    #Re-assign previous state
    x_prev = xk
    
    # Initialize Operating Point T
    algopt = np.array([0,0,0,0,0,0])
    if k == 0:
        Top = SE3.exp(SE3.wedge(np.array([0,0,0,0,0,0])))
    algoptprev = None
    #----- Point Cloud Alignment, iterative optimization for each time step k -------
    counter = 0
    while algoptprev is None or ca.norm_2(algopt-algoptprev)>1e-8:    
        algoptprev = algopt
        algopt = barfoot_solve(Top,landmarks,yk,assoc,weight)
        Top = SE3.exp(SE3.wedge(algopt))@Top
        counter +=1
    #--------------------------------------
    Top_allbody.append(Top)
    
    #Append "true state" history
    hist['x'].append(xk)
    hist['R'].append(Rotk)
    hist['T'].append(Tk)
    hist['Frame'].append(k)
    for yi in yk:
        hist['y'].append(np.hstack([yi, k]))      # History of measurements recorded at each time step
    for i in range(len(assoc)):
        hist['assoc'].append(np.hstack(['p'+str(assoc[i]), 'x'+str(k)]))
 
    
for key in hist.keys():
    hist[key] = np.array(hist[key])


tf =  time.time() - t0
print('runtime [s]: ', tf)

runtime [s]:  0.18142199516296387


In [21]:
print(Top_allbody[1])
print(hist['T'][1])
counter

@1=0.605152, @2=0, @3=1, 
[[@1, 0.79611, @2, -6.78886], 
 [-0.79611, @1, @2, 2.35311], 
 [@2, @2, @3, @2], 
 [@2, @2, @2, @3]]
[[ 0.60181502  0.79863551  0.         -6.80544507]
 [-0.79863551  0.60181502  0.          2.39590653]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


5